In [1]:
from src.utils import *
from src.graph_model import *
from src.owl2vec import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


In [3]:
#dataset_name = 'family'
#dataset_name = 'pizza'
dataset_name = 'OWL2DL-1'
#dataset_name = 'lubm'

In [4]:
experiments = get_experimets(dataset_name)

In [5]:
for experiment in experiments[-1:]: 
    dataset_name = experiment['dataset_name']
    file_name = experiment['file_name']
    format_ = experiment['format_']
    add_noise = experiment['add_noise']
          
    model = OWL2Vec(file_name         = file_name,
                    dataset_name      = dataset_name,
                    kge_model         = 'transe',
                    emb_dim           = 256,
                    margin            = 0.1,
                    weight_decay      = 0.0,
                    batch_size        = 4096*8,
                    lr                = 0.0001,
                    num_negs          = 4,
                    test_batch_size   = 32,
                    epochs            = 500,
                    device            = device,
                    seed              = 42,
                    initial_tolerance = 5)

    model.train()
    
    print(f'{file_name}:')
    subsumption_metrics, membership_metrics = model.test()
    print()

org.semanticweb.owlapi.model.OWLOntologyAlreadyExistsException: org.semanticweb.owlapi.model.OWLOntologyAlreadyExistsException: Ontology already exists. OntologyID(OntologyIRI(<https://kracr.iiitd.edu.in/OWL2Bench>) VersionIRI(<null>))

In [6]:
dataset_name

'OWL2DL-1'

In [7]:
dataset = PathDataset('datasets/bin/OWL2DL-1_train.owl', validation_path = 'datasets/bin/OWL2DL-1_val.owl', testing_path = 'datasets/bin/OWL2DL-1_test.owl')

org.semanticweb.owlapi.model.OWLOntologyAlreadyExistsException: org.semanticweb.owlapi.model.OWLOntologyAlreadyExistsException: Ontology already exists. OntologyID(OntologyIRI(<https://kracr.iiitd.edu.in/OWL2Bench>) VersionIRI(<null>))